<a href="https://colab.research.google.com/github/AiJared/Advanced/blob/master/StyleGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import keras

In [ ]:
from google.colab import drive
drive.mount('/content/MyDrive')

Mounted at /content/MyDrive


In [ ]:
class StyleGAN:
  def __init__(self, img_shape=(256, 256, 3), latent_dim=512, n_styles=18):
    """
    Initialize stylegan with specific parameters

    Args:
      img_shape(tuple): Dimensions of output images
      latent_dim(int): Dimensionality of the input latent space
      n_styles(int): Number of style layers
    """

    self.img_shape = img_shape
    self.latent_dim = latent_dim
    self.n_styles = n_styles

    # Key StyleGAN Components
    self.mapping_network = self.build_mapping_network()
    self.synthesis_network = self.build_synthesis_network()
    self.discriminator = self.build_discriminator()

    # Compile the full generator and adversarial model
    self.generator = self.build_generator()
    self.adversarial_model = self.build_adversarial_model()

  def build_mapping_network(self):
    """
    Mapping network: Transforms input noise to intermediate latent space
    Key styleGAN Innovation: Non-linear transformation of input noise
    """

    model  = keras.Sequential([
        keras.layers.Dense(self.latent_dim, activation='relu'),
        keras.layers.Dense(self.latent_dim, activation='relu'),
        keras.layers.Dense(self.latent_dim, activation='relu')
    ])

    return model

  def build_style_block(self, out_channels, upsample=True):
    """
    Create a style block with Adaptive Instance Normalization (Adain) concept.

    Args:
      out_channels (int): Number of output channels
      upsample (bool): Whether to upsample the feature map

    Returns:
      keras.Model: Style block model
    """

    block = keras.Sequential()

    if upsample:
      # upsampling layer
      block.add(keras.layers.UpSampling2D(size=(2,2)))

    # convolutional layer
    block.add(keras.layers.Conv2D(out_channels, 3, padding='same'))

    # Noise input (Simulated AdaIN)
    block.add(keras.layers.Lambda(lambda x: x + tf.random.normal(tf.shape(x)) * 0.1))

    # Activate
    block.add(keras.layers.LeakyReLU(0.2))

    return block
  def build_synthesis_network(self):
    """
    synthesis network: Generates images from style vectors
    Progressive growing of feature maps
    """

    model = keras.Sequential([
        # starting block
        keras.layers.Dense(4 * 4 * 512, input_shape=(self.latent_dim,)),
        keras.layers.Reshape((4, 4, 512)),

        # Progressive style blocks
        self.build_style_block(256), # 8x8
        self.build_style_block(128), # 16x16
        self.build_style_block(64), # 32x32
        self.build_style_block(16), # 64x64

        # Final convolution layer to match image channels
        keras.layers.Conv2D(self.img_shape[2], 1, activation='tanh')
    ])

    return model

  def build_discriminator(self):
    """

    Discriminator Network: Distinguishes real from generated images
    Uses multi-scale feature matching
    """

    model = keras.Sequential([
        # multi-scale feature extraction
        keras.layers.Conv2D(64, 4, strides=2, padding='same', input_shape=self.img_shape),
        keras.layers.LeakyReLU(0.2),

        keras.layers.Conv2D(128, 4, strides=2, padding='same'),
        keras.layers.BatchNormalization(),
        keras.layers.LeakyReLU(0.2),

        keras.layers.Conv2D(256, 4, strides=2, padding='same'),
        keras.layers.BatchNormalization(),
        keras.layers.LeakyReLU(0.2),

        keras.layers.Flatten(),
        keras.layers.Dense(1, activation='sigmoid')
    ])

    return model

  def build_generator(self):
    """
    Full Generator: Combines mapping and synthesis networks.
    """

    def generator(noise):
      # transform noise through mapping network
      w = self.mapping_network(noise)

      # generate image through synthesis network
      image = self.synthesis_network(w)
      return image

    return generator

  def build_adversarial_model(self):
    """
    Build generator-discriminator model
    """

    self.discriminator.trainable = False

    noise_input = keras.Input(shape=(self.latent_dim,))
    generated_image = self.generator(noise_input)
    validity = self.discriminator(generated_image)

    model = keras.Model(noise_input, validity)

    return model

  def train_step(self, real_images):
    """
    Single training step for stylegan
    Implements adversarial training with gradient computation
    """

    batch_size = tf.shape(real_images)[0]

    # Generate random noise
    noise = tf.random.normal([batch_size, self.latent_dim])

    # Generate fake images
    generated_images = self.generator(noise)

    # Prepare labels
    real_labels = tf.ones((batch_size, 1))
    fake_labels = tf.zeros((batch_size, 1))

    # Train Discriminator
    with tf.GradientTape() as disc_tape:
      real_predictions = self.discriminator(real_images)
      fake_predictions = self.discriminator(generated_images)

      disc_real_loss = keras.losses.binary_crossentropy(real_labels, real_predictions)
      disc_fake_loss = keras.losses.binary_crossentorpy(fake_labels, fake_predictions)

      disc_loss = 0.5 * (disc_real_loss + disc_fake_loss)
    # Compute and apply discriminator gradients
    disc_gradients = disc_tape.gradient(disc_loss, self.discriminator.trainable_variables)
    keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5).apply_gradients(
        zip(disc_gradients, self.discriminator.trainable_variables)
    )

    # Train Generator
    with tf.GradientTape() as gen_tape:
      noise = tf.random.normal([batch_size, self.latent_dim])
      generated_images = self.generator(noise)

      fake_predictions = self.discriminator(generated_images)
      gen_loss = keras.losses.binary_crossentropy(real_labels, fake_predictions)

    #Compute and apply generator gradients
    gen_gradients = gen_tape.gradient(gen_loss,
        self.mapping_network.trainable_variables +
        self.synthesis_network.trainable_variables
    )

    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)
    optimizer.apply_gradients(zip(gen_gradients,
        self.mapping_network.trainable_variables +
        self.synthesis_network.trainable_variables
    ))

    return disc_loss, gen_loss

  def train(self, dataset, epochs=200):
    """
    Train the stylegan

    Args:
      dataset (tf.data.Dataset): Training dataset
      epochs (int): Number of training epochs
    """
    for epoch in range(epochs):
      for batch in dataset:
        disc_loss, gen_loss = self.train_step(batch)

    # Generate and save images periodically one by one
    if epoch % 10 == 0:
      self.generate_and_save_images(epoch)
      print(f"Epoch {epoch}: Discriminator Loss: {disc_loss.numpy()}, Generator Loss: {gen_loss.numpy()}")

  def generate_and_save_images(self, epoch, num_examples=16):
        """
        Generate sample images during training

        Args:
            epoch (int): Current training epoch
            num_examples (int): Number of images to generate
        """
        noise = tf.random.normal([num_examples, self.latent_dim])
        generated_images = self.generator(noise)

        plt.figure(figsize=(10,10))
        for i in range(num_examples):
          plt.subplot(4, 4, i+1)
          # Rescale images from [-1,1] to [0,1]
          plt.imshow((generated_images[i] + 1) / 2.0)
          plt.axis('off')
